In [ ]:
import sys
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager # Added import

# NOTE: `!pip install selenium` has been moved to a separate cell for cleaner execution.

def ekran_goruntusu_al(url, dosya_adi):
    # Colab için Chrome Ayarları (Burası Kritiktir)
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') # Tarayıcıyı gizli modda çalıştır
    options.add_argument('--no-sandbox') # Colab güvenliği için gerekli
    options.add_argument('--disable-dev-shm-usage') # Bellek sorunlarını önler
    options.add_argument('--window-size=1920,1080') # Ekran çözünürlüğü

    # Tarayıcıyı Başlat
    # Use ChromeDriverManager to automatically get the correct chromedriver
    service = Service(ChromeDriverManager().install()) # Modified this line
    driver = webdriver.Chrome(service=service, options=options)

    try:
        print(f"{url} adresine gidiliyor...")
        driver.get(url)

        # Gratis.com dinamik olduğu için görsellerin yüklenmesi adına biraz beklemeli
        # veya sayfayı aşağı kaydırmalıyız.
        time.sleep(5)

        # Sayfayı biraz aşağı kaydır (Lazy load tetiklemek için)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 3);")
        time.sleep(2)

        # Ekran görüntüsünü kaydet
        driver.save_screenshot(f"{dosya_adi}.png")
        print(f"Ekran görüntüsü kaydedildi: {dosya_adi}.png")

    except Exception as e:
        print(f"Hata oluştu: {e}")
    finally:
        driver.quit()

# Örnek kullanım: Sadece bir ürün veya kategori sayfası için
ekran_goruntusu_al("https://www.gratis.com/cilt-bakim", "gratis_cilt_bakim")

In [ ]:
%%shell
# 1. Sistem paketlerini güncelle
sudo apt -y update

# 2. Google Chrome'un kararlı sürümünü indir ve kur
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
sudo apt install ./google-chrome-stable_current_amd64.deb -y

# 3. Gerekli Python kütüphanelerini kur
pip install selenium webdriver-manager

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from IPython.display import Image, display

# --- AYARLAR ---
HEDEF_URL = "https://www.gratis.com/sac-bakim"
KAYIT_ISMI = "gratis_sayfasi.png"

# Chrome Ayarları (Colab için optimize edildi)
chrome_options = Options()
chrome_options.add_argument('--headless')  # Arayüzsüz mod
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--window-size=1920,1080')

try:
    print("Sürücü yükleniyor ve tarayıcı başlatılıyor...")
    # ChromeDriverManager ile otomatik sürücü yönetimi
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    print(f"Siteye gidiliyor: {HEDEF_URL}")
    driver.get(HEDEF_URL)

    # Sayfanın ilk yüklenmesi için bekle
    time.sleep(5)

    # --- LAZY LOAD (AŞAĞI KAYDIRMA) ---
    print("Ürünlerin yüklenmesi için sayfa kaydırılıyor...")
    for i in range(1, 5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight * arguments[0] / 5);", i)
        time.sleep(2)

    # Ekran görüntüsünü al
    driver.save_screenshot(KAYIT_ISMI)
    print(f"Başarılı! Görüntü kaydedildi: {KAYIT_ISMI}")

    # Ekrana bas
    display(Image(filename=KAYIT_ISMI))

except Exception as e:
    print(f"Hata oluştu: {e}")

finally:
    # Eğer driver tanımlandıysa kapat
    if 'driver' in locals():
        driver.quit()

In [ ]:
import json
import pandas as pd
from google.colab import files
import sys

# 1. ADIM: Dosya Yükleme
print("Lütfen dosyanızı tekrar yükleyin:")
uploaded = files.upload()
dosya_ismi = next(iter(uploaded))

def json_ayiklayici(text):
    """
    Metin içindeki products:[...] dizisini akıllıca bulup ayıklar.
    """
    # Farklı varyasyonları dene
    markers = ['"products":[', '\\"products\\":[', 'products:[']
    start_index = -1

    for marker in markers:
        start_index = text.find(marker)
        if start_index != -1:
            # Marker'ın bittiği yer (köşeli parantezin başı)
            start_index = start_index + len(marker) - 1
            break

    if start_index == -1:
        return None

    # Parantez sayma mantığı (Bracket Counting)
    # Bu yöntem iç içe geçmiş tüm yapıları hatasız alır
    count = 0
    list_str = ""
    started = False

    for i in range(start_index, len(text)):
        char = text[i]

        if char == '[':
            count += 1
            started = True
        elif char == ']':
            count -= 1

        list_str += char

        # Başladıysa ve sayaç sıfıra döndüyse liste bitmiştir
        if started and count == 0:
            return list_str

    return None

try:
    # 2. ADIM: Dosyayı Oku
    with open(dosya_ismi, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()

    print(f"\nDosya uzunluğu: {len(content)} karakter.")
    print("Veri bloğu aranıyor...")

    # 3. ADIM: Veriyi Ayıkla
    raw_json = json_ayiklayici(content)

    if raw_json:
        print("Veri bloğu bulundu! Temizleniyor...")

        # Next.js verileri bazen string içinde string (escaped) olur.
        # Bunu düzeltmek için:
        if raw_json.startswith('[') and raw_json.count('\\"') > raw_json.count('"') / 2:
             # Yoğun şekilde escape edilmişse temizle
             raw_json = raw_json.replace('\\"', '"').replace('\\\\', '\\')

        try:
            products = json.loads(raw_json)
            print(f"\n>>> BAŞARILI: {len(products)} adet ürün ayrıştırıldı.")

            # 4. ADIM: Excel'e Dönüştür
            veriler = []
            for item in products:
                # Olası eksik alanları yönet
                attr = item.get('attributes', {})
                prices = item.get('prices', {})

                # Resim linkini güvenli al
                img = "Yok"
                if item.get('imageUrls') and len(item['imageUrls']) > 0:
                    img = item['imageUrls'][0].get('fileUrl', 'Yok')

                veriler.append({
                    'Marka': attr.get('brand'),
                    'Ürün Adı': attr.get('displayName'),
                    'Normal Fiyat': prices.get('normalPriceLabel'),
                    'Satış Fiyatı': prices.get('promotionPriceLabel') or prices.get('salesPriceLabel'),
                    'Stok': item.get('stockStatus'),
                    'Link': f"https://www.gratis.com{item.get('shareLink')}" if item.get('shareLink') else "",
                    'Resim': img
                })

            df = pd.DataFrame(veriler)
            excel_name = "gratis_sonuc.xlsx"
            df.to_excel(excel_name, index=False)

            print(f"Tablo oluşturuldu. '{excel_name}' indiriliyor...")
            print(df.head(3)) # İlk 3 satırı göster
            files.download(excel_name)

        except json.JSONDecodeError as e:
            print(f"HATA: Bulunan veri JSON formatına çevrilemedi. Hata detayı:\n{e}")
            print("Bulunan verinin başı:", raw_json[:100])

    else:
        print("HATA: 'products':[...] yapısı dosya içinde bulunamadı.")
        print("Lütfen Postman'de 'Pretty' değil 'Raw' sekmeye geçip kopyaladığınızdan emin olun.")

except Exception as e:
    print(f"Beklenmeyen bir hata oluştu: {e}")

In [ ]:
import requests
import json
import pandas as pd
import time
import re
from google.colab import files

# --- AYARLAR ---
# Hangi kategoriyi çekmek istiyorsunuz? (Tarayıcıdaki linki buraya yapıştırın)
BASE_URL = "https://www.gratis.com/makyaj-c-501"

# Kaç sayfa çekilsin? (Örn: 1'den 10'a kadar)
BASLANGIC_SAYFA = 1
BITIS_SAYFA = 5

# --- HEADER AYARLARI (ÇOK ÖNEMLİ) ---
# Postman'de "Headers" sekmesindeki değerleri buraya eklemelisiniz.
# Yoksa site sizin robot olduğunuzu anlar ve veri vermez.
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    # Postman'den aldığınız 'Cookie' değerini buraya yapıştırırsanız daha garanti olur.
    # "Cookie": "buraya_postman_cookie_degerini_yapistirin"
}

def json_ayiklayici(text):
    """HTML içinden JSON verisini bulur"""
    markers = ['"products":[', '\\"products\\":[', 'products:[']
    start_index = -1
    for marker in markers:
        start_index = text.find(marker)
        if start_index != -1:
            start_index = start_index + len(marker) - 1
            break

    if start_index == -1: return None

    count = 0
    list_str = ""
    started = False
    for i in range(start_index, len(text)):
        char = text[i]
        if char == '[':
            count += 1
            started = True
        elif char == ']':
            count -= 1
        list_str += char
        if started and count == 0:
            return list_str
    return None

# --- ANA DÖNGÜ ---
tum_urunler = []

print(f"Veri çekme işlemi başlıyor... ({BASLANGIC_SAYFA}. sayfadan {BITIS_SAYFA}. sayfaya kadar)")

for sayfa in range(BASLANGIC_SAYFA, BITIS_SAYFA + 1):
    # Sayfa parametresini ekle (Örn: ?page=2)
    url = f"{BASE_URL}?page={sayfa}"
    print(f"Sayfa {sayfa} taranıyor... ({url})")

    try:
        response = requests.get(url, headers=HEADERS)

        if response.status_code != 200:
            print(f"UYARI: Sayfa {sayfa} açılamadı. Kod: {response.status_code}")
            continue

        raw_json = json_ayiklayici(response.text)

        if raw_json:
            # Temizlik
            if raw_json.startswith('[') and raw_json.count('\\"') > raw_json.count('"') / 2:
                 raw_json = raw_json.replace('\\"', '"').replace('\\\\', '\\')

            products = json.loads(raw_json)
            print(f"   -> {len(products)} adet ürün bulundu.")

            for item in products:
                attr = item.get('attributes', {})
                prices = item.get('prices', {})

                img = "Yok"
                if item.get('imageUrls') and len(item['imageUrls']) > 0:
                    img = item['imageUrls'][0].get('fileUrl', 'Yok')

                tum_urunler.append({
                    'Sayfa No': sayfa,
                    'Marka': attr.get('brand'),
                    'Ürün Adı': attr.get('displayName'),
                    'Normal Fiyat': prices.get('normalPriceLabel'),
                    'Satış Fiyatı': prices.get('promotionPriceLabel') or prices.get('salesPriceLabel'),
                    'Stok': item.get('stockStatus'),
                    'Link': f"https://www.gratis.com{item.get('shareLink')}" if item.get('shareLink') else "",
                    'Resim': img
                })
        else:
            print(f"   -> Sayfa {sayfa}'da veri bulunamadı.")

    except Exception as e:
        print(f"   -> Hata oluştu: {e}")

    # Siteyi yormamak için her sayfada 2 saniye bekle
    time.sleep(2)

# --- KAYDETME ---
if tum_urunler:
    df = pd.DataFrame(tum_urunler)
    dosya_adi = "gratis_tum_urunler.xlsx"
    df.to_excel(dosya_adi, index=False)
    print(f"\nTOPLAM {len(tum_urunler)} ürün başarıyla çekildi ve kaydedildi!")
    files.download(dosya_adi)
else:
    print("\nHiçbir veri çekilemedi. Header ayarlarını kontrol edin.")

In [ ]:
# Gerekli araçları yüklüyoruz
!pip install selenium webdriver-manager pandas
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import sys
import json
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# --- AYARLAR ---
BASLANGIC_SAYFA = 1
BITIS_SAYFA = 3  # DİKKAT: Test için 3 yaptık. Tüm ürünler için burayı 195 yapın.
BASE_URL = "https://www.gratis.com/makyaj-c-501?page={}"

# --- TARAYICI AYARLARI ---
chrome_options = Options()
chrome_options.add_argument('--headless') # Tarayıcıyı gizli çalıştır
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

# --- YARDIMCI FONKSİYON: SİZİN İÇİN YAZDIĞIMIZ AKILLI AYIKLAYICI ---
def json_ayiklayici(text):
    # Bu fonksiyon, HTML içindeki karmaşık ürün verisini cımbızla çeker
    markers = ['"products":[', '\\"products\\":[', 'products:[']
    start_index = -1
    for marker in markers:
        start_index = text.find(marker)
        if start_index != -1:
            start_index = start_index + len(marker) - 1
            break

    if start_index == -1: return None

    count = 0
    list_str = ""
    started = False

    for i in range(start_index, len(text)):
        char = text[i]
        if char == '[':
            count += 1
            started = True
        elif char == ']':
            count -= 1
        list_str += char
        if started and count == 0:
            return list_str
    return None

# --- ANA PROGRAM ---
tum_urunler = []
print(f"Veri çekme işlemi başlıyor... Hedef: {BITIS_SAYFA - BASLANGIC_SAYFA + 1} sayfa taranacak.\n")

for i in range(BASLANGIC_SAYFA, BITIS_SAYFA + 1):
    url = BASE_URL.format(i)
    print(f"Sayfa {i} taranıyor... ", end="")

    try:
        driver.get(url)
        time.sleep(3) # Sayfanın yüklenmesi için bekle (İnternet hızına göre artırılabilir)

        # Sayfa kaynağını al
        page_source = driver.page_source

        # Veriyi ayıkla
        raw_json = json_ayiklayici(page_source)

        if raw_json:
            # Temizlik (Next.js escape karakterlerini düzelt)
            if raw_json.startswith('[') and raw_json.count('\\"') > raw_json.count('"') / 2:
                 raw_json = raw_json.replace('\\"', '"').replace('\\\\', '\\')

            products = json.loads(raw_json)
            print(f"✅ {len(products)} ürün bulundu.")

            # Ürünleri listeye ekle
            for item in products:
                attr = item.get('attributes', {})
                prices = item.get('prices', {})
                img = item['imageUrls'][0].get('fileUrl', 'Yok') if item.get('imageUrls') else "Yok"

                tum_urunler.append({
                    'Sayfa': i,
                    'Marka': attr.get('brand'),
                    'Ürün Adı': attr.get('displayName'),
                    'Normal Fiyat': prices.get('normalPriceLabel'),
                    'Satış Fiyatı': prices.get('promotionPriceLabel') or prices.get('salesPriceLabel'),
                    'Stok': item.get('stockStatus'),
                    'Resim': img,
                    'Link': f"https://www.gratis.com{item.get('shareLink')}"
                })
        else:
            print("❌ Veri bulunamadı.")

    except Exception as e:
        print(f"⚠️ Hata: {e}")

# --- KAYDETME ---
driver.quit()
print(f"\nToplam {len(tum_urunler)} ürün toplandı.")

if tum_urunler:
    df = pd.DataFrame(tum_urunler)
    excel_adi = "gratis_tum_urunler.xlsx"
    df.to_excel(excel_adi, index=False)
    print(f"Dosya oluşturuldu: {excel_adi}")

    from google.colab import files
    files.download(excel_adi)
else:
    print("Hiç veri toplanamadı.")

In [ ]:
import sys
import json
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from google.colab import files

# --- AYARLAR (TÜM ÜRÜNLER İÇİN GÜNCELLENDİ) ---
BASLANGIC_SAYFA = 1
BITIS_SAYFA = 195  # 4594 ürün yaklaşık 192 sayfa yapar, garanti olsun diye 195 yaptık.
BASE_URL = "https://www.gratis.com/makyaj-c-501?page={}"

# --- TARAYICI AYARLARI ---
print("Tarayıcı başlatılıyor...")
chrome_options = Options()
chrome_options.add_argument('--headless') # Arka planda çalışır
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# Tarayıcıyı kandırmak için User-Agent ekliyoruz
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
driver = webdriver.Chrome(options=chrome_options)

# --- YARDIMCI FONKSİYON ---
def json_ayiklayici(text):
    # HTML içinden JSON verisini cımbızla çeker
    markers = ['"products":[', '\\"products\\":[', 'products:[']
    start_index = -1
    for marker in markers:
        start_index = text.find(marker)
        if start_index != -1:
            start_index = start_index + len(marker) - 1
            break

    if start_index == -1: return None

    count = 0
    list_str = ""
    started = False

    for i in range(start_index, len(text)):
        char = text[i]
        if char == '[':
            count += 1
            started = True
        elif char == ']':
            count -= 1
        list_str += char
        if started and count == 0:
            return list_str
    return None

# --- ANA DÖNGÜ ---
tum_urunler = []

print(f"\n🚀 Veri çekme işlemi başlıyor... Hedef: {BASLANGIC_SAYFA}. sayfadan {BITIS_SAYFA}. sayfaya kadar.")
print("Bu işlem ürün sayısına göre 10-15 dakika sürebilir. Lütfen bekleyin...\n")

baslangic_zamani = time.time()

for i in range(BASLANGIC_SAYFA, BITIS_SAYFA + 1):
    url = BASE_URL.format(i)

    # İlerleme çubuğu gibi çıktı verelim
    sys.stdout.write(f"\rSayfa {i}/{BITIS_SAYFA} taranıyor... ")
    sys.stdout.flush()

    try:
        driver.get(url)
        # Sayfanın yüklenmesi için dinamik bekleme
        time.sleep(1.5)

        page_source = driver.page_source
        raw_json = json_ayiklayici(page_source)

        if raw_json:
            # Temizlik
            if raw_json.startswith('[') and raw_json.count('\\"') > raw_json.count('"') / 2:
                 raw_json = raw_json.replace('\\"', '"').replace('\\\\', '\\')

            try:
                products = json.loads(raw_json)

                # Eğer sayfada ürün yoksa (boş liste) döngüyü bitir
                if len(products) == 0:
                    print(f"\n⚠️ Sayfa {i}'de ürün bulunamadı. İşlem erken bitiriliyor.")
                    break

                for item in products:
                    attr = item.get('attributes', {})
                    prices = item.get('prices', {})
                    img = item['imageUrls'][0].get('fileUrl', 'Yok') if item.get('imageUrls') else "Yok"

                    tum_urunler.append({
                        'Sayfa No': i,
                        'Marka': attr.get('brand'),
                        'Ürün Adı': attr.get('displayName'),
                        'Normal Fiyat': prices.get('normalPriceLabel'),
                        'Satış Fiyatı': prices.get('promotionPriceLabel') or prices.get('salesPriceLabel'),
                        'Stok': item.get('stockStatus'),
                        'Kategori': attr.get('categories', [''])[0] if attr.get('categories') else '',
                        'Resim': img,
                        'Link': f"https://www.gratis.com{item.get('shareLink')}"
                    })
            except:
                pass # JSON hatası olursa geç
        else:
            # Veri bulunamazsa muhtemelen son sayfaya geldik veya engellendik
            # 2 kez üst üste veri bulamazsa durdurabiliriz ama şimdilik devam etsin
            pass

    except Exception as e:
        print(f"\n⚠️ Sayfa {i} hatası: {e}")

# --- KAYDETME ---
driver.quit()
gecen_sure = (time.time() - baslangic_zamani) / 60

print(f"\n\n✅ İŞLEM TAMAMLANDI!")
print(f"Toplam Süre: {gecen_sure:.1f} dakika")
print(f"Toplam Toplanan Ürün: {len(tum_urunler)}")

if tum_urunler:
    df = pd.DataFrame(tum_urunler)

    # Excel dosya ismi
    excel_adi = "gratis_tum_urunler_tam_liste.xlsx"

    print("Excel dosyası oluşturuluyor...")
    df.to_excel(excel_adi, index=False)

    print(f"İndirme başlatılıyor: {excel_adi}")
    files.download(excel_adi)
else:
    print("Hiç ürün çekilemedi. Lütfen bağlantınızı kontrol edin.")

In [ ]:
%%shell
# Gerekli Linux ve Python paketlerini kur
sudo apt -y update
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
sudo apt install ./google-chrome-stable_current_amd64.deb -y
pip install selenium webdriver-manager pandas openpyxl

In [ ]:
import sys
import json
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from google.colab import files

# --- 1. HEDEFLENEN KATEGORİLER ---
# Linklerin sonundaki "?page=" kısmını kod otomatik ekleyecek.
KATEGORILER = {
    "Makyaj": "https://www.gratis.com/makyaj-c-501",
    "Cilt Bakım": "https://www.gratis.com/cilt-bakim-c-502",
    "Saç Bakım": "https://www.gratis.com/sac-bakim-c-503",
    "Parfüm & Deodorant": "https://www.gratis.com/parfum-deodorant-c-504",
    "Erkek Bakım": "https://www.gratis.com/erkek-bakim-c-505",
    "Kişisel Bakım": "https://www.gratis.com/kisisel-bakim-c-506",
    "Anne & Bebek": "https://www.gratis.com/anne-bebek-c-507",
    "Ev & Yaşam": "https://www.gratis.com/ev-yasam-c-508",
    "Moda & Aksesuar": "https://www.gratis.com/moda-aksesuar-c-509",
    "Süpermarket": "https://www.gratis.com/supermarket-c-510",
    "Elektrikli Ürünler": "https://www.gratis.com/elektrikli-urunler-c-511"
}

# --- 2. TARAYICI AYARLARI ---
print("🚀 Tarayıcı hazırlanıyor...")
chrome_options = Options()
chrome_options.add_argument('--headless') # Arka planda çalışır
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("window-size=1920,1080")
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
except Exception as e:
    print(f"HATA: Tarayıcı başlatılamadı. {e}")
    sys.exit()

# --- 3. JSON AYIKLAYICI FONKSİYON ---
def json_ayiklayici(text):
    """HTML içindeki gizli ürün verisini bulur"""
    markers = ['"products":[', '\\"products\\":[', 'products:[']
    start_index = -1
    for marker in markers:
        start_index = text.find(marker)
        if start_index != -1:
            start_index = start_index + len(marker) - 1
            break

    if start_index == -1: return None

    count = 0
    list_str = ""
    started = False

    for i in range(start_index, len(text)):
        char = text[i]
        if char == '[':
            count += 1
            started = True
        elif char == ']':
            count -= 1
        list_str += char
        if started and count == 0:
            return list_str
    return None

# --- 4. ANA DÖNGÜ ---
tum_urunler_listesi = []
baslangic_zamani = time.time()

print(f"\nToplam {len(KATEGORILER)} kategori taranacak.")

for kategori_adi, base_url in KATEGORILER.items():
    print(f"\n>>> KATEGORİ BAŞLIYOR: {kategori_adi.upper()}")
    sayfa = 1

    while True:
        # Sayfa limitini abartmamak için güvenlik (Örn: Max 200 sayfa)
        if sayfa > 200:
            break

        url = f"{base_url}?page={sayfa}"

        try:
            driver.get(url)
            time.sleep(1.5) # Sayfa yüklenmesi için bekle

            page_source = driver.page_source
            raw_json = json_ayiklayici(page_source)

            if raw_json:
                # Temizlik
                if raw_json.startswith('[') and raw_json.count('\\"') > raw_json.count('"') / 2:
                     raw_json = raw_json.replace('\\"', '"').replace('\\\\', '\\')

                try:
                    products = json.loads(raw_json)

                    # Eğer ürün listesi boşsa bu kategori bitmiş demektir
                    if len(products) == 0:
                        print(f"   -> Sayfa {sayfa}: Ürün yok. Kategori tamamlandı.")
                        break

                    # Ürünleri işle
                    for item in products:
                        attr = item.get('attributes', {})
                        prices = item.get('prices', {})
                        img = item['imageUrls'][0].get('fileUrl', 'Yok') if item.get('imageUrls') else "Yok"

                        tum_urunler_listesi.append({
                            'Ana Kategori': kategori_adi,
                            'Sayfa': sayfa,
                            'Marka': attr.get('brand'),
                            'Ürün Adı': attr.get('displayName'),
                            'Normal Fiyat': prices.get('normalPriceLabel'),
                            'Satış Fiyatı': prices.get('promotionPriceLabel') or prices.get('salesPriceLabel'),
                            'Stok': item.get('stockStatus'),
                            'Ürün ID': item.get('id'),
                            'Resim Linki': img,
                            'Ürün Linki': f"https://www.gratis.com{item.get('shareLink')}"
                        })

                    # İlerleme durumunu yazdır
                    sys.stdout.write(f"\r   -> Sayfa {sayfa}: {len(products)} ürün alındı. (Toplam: {len(tum_urunler_listesi)})")
                    sys.stdout.flush()

                    sayfa += 1

                except json.JSONDecodeError:
                    print(f"\n   -> HATA: Sayfa {sayfa} JSON okunamadı, geçiliyor.")
                    sayfa += 1
            else:
                # JSON bulunamadıysa muhtemelen sayfa yoktur
                print(f"\n   -> Sayfa {sayfa}: Veri bulunamadı. Kategori sonu.")
                break

        except Exception as e:
            print(f"\n   -> Beklenmedik Hata (Sayfa {sayfa}): {e}")
            break

# --- 5. KAYDETME ---
driver.quit()
gecen_sure = (time.time() - baslangic_zamani) / 60

print(f"\n\n{'='*40}")
print(f"✅ TÜM İŞLEMLER TAMAMLANDI!")
print(f"Toplam Süre: {gecen_sure:.1f} dakika")
print(f"Toplam Toplanan Ürün: {len(tum_urunler_listesi)}")
print(f"{'='*40}")

if tum_urunler_listesi:
    df = pd.DataFrame(tum_urunler_listesi)
    dosya_adi = "gratis_tum_kategoriler_full.xlsx"

    print("Excel dosyası oluşturuluyor...")
    df.to_excel(dosya_adi, index=False)

    print(f"İndiriliyor: {dosya_adi}")
    files.download(dosya_adi)
else:
    print("Hiç veri toplanamadı.")